- cropping to focus on players
- detect wide views and use crops for them

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Imports

In [ ]:
import os
import re
import cv2
import time
import torch
import imageio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from tqdm.notebook import tqdm
from skimage.transform import resize

In [ ]:
from params import *

## Data

### Load

In [ ]:
df_train = pd.read_csv(DATA_PATH + 'df_train.csv')

In [ ]:
frame_impacts = df_train[["image_name", "extended_impact"]].groupby('image_name').max().reset_index()
frame_impacts = frame_impacts.rename(columns={"extended_impact": "frame_has_impact"})
df_train = df_train.merge(frame_impacts, on="image_name")

In [ ]:
# df_train = df_train[df_train['frame_has_impact'] == 1]
df_train['extended_impact'] += 1 
df_train['impact'] +=1 

In [ ]:
folds = pd.read_csv(OUT_DIR + "folds.csv")
df_train = df_train.merge(folds, on="video")

In [ ]:
img_path = IMG_PATH_F

In [ ]:
if "512" in img_path:
    df_train["x"] = (df_train["x"] / IMG_SHAPE[1] * SIZE).astype(int)
    df_train["y"] = (df_train["y"] / IMG_SHAPE[0] * SIZE).astype(int)
    df_train["w"] = (df_train["w"] / IMG_SHAPE[1] * SIZE).astype(int)
    df_train["h"] = (df_train["h"] / IMG_SHAPE[0] * SIZE).astype(int)

## Dataset

In [ ]:
from data.dataset import NFLDatasetDet
from data.transforms import get_transfos_det
from utils.plot import plot_bboxes

In [ ]:
dataset = NFLDatasetDet(
    df_train.copy(),
    transforms=get_transfos_det(train=True, visualize=True),
    root=img_path,
)

In [ ]:
image, boxes, label, vid, frame = dataset[0]
boxes = boxes.int()

plt.figure(figsize=(8, 8))
plot_bboxes(image.numpy().transpose(1, 2, 0).copy(), boxes, label, transpose=True)
plt.show()

In [ ]:
image.size()

## Model

In [ ]:
from model_zoo.models_det import get_model, get_val_model

In [ ]:
name = 'tf_efficientdet_d0'

In [ ]:
model = get_model(
    name,
    num_classes=2
)

In [ ]:
dataset = NFLDatasetDet(
    df_train,
    transforms=get_transfos_det(train=True),
    root=img_path,
)

In [ ]:
image, boxes, target = dataset[0][:3]

In [ ]:
pred = model(
    image.unsqueeze(0),
    boxes.unsqueeze(0),
    target.unsqueeze(0),
)

In [ ]:
val_model = get_val_model(model)

In [ ]:
scales = torch.tensor([1]).long()

pred_val = val_model(
    image.unsqueeze(0),
    scales
)

In [ ]:
pred_val.shape

## Training

In [ ]:
from training.main_det import k_fold_det

In [ ]:
from utils.logger import prepare_log_folder, save_config, create_logger

In [ ]:
BATCH_SIZES = {
    "tf_efficientdet_d0": 12,
    "tf_efficientdet_d1": 8,
#     "tf_efficientdet_d2": 8,
    "tf_efficientdet_d3": 6,
    "tf_efficientdet_d4": 4,
    "tf_efficientdet_d5": 2,
    "tf_efficientdet_d6": 2,
}

In [ ]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1
    img_path = img_path
    device = "cuda" if torch.cuda.is_available() else "cpu"
    save_weights = True


    # k-fold
    k = 5
    random_state = 0
    selected_folds = [0] # , 1, 2, 3, 4

    # Model
    name = "tf_efficientdet_d4"
    num_classes = 1

    # Training        
    optimizer = "Adam"
    batch_size = BATCH_SIZES[name]  # TODO : VERIF ACC STEPS
    acc_steps = 1
    epochs = 15
    swa_first_epoch = 20

    lr = 5e-4  # 5e-4 / 1e-3
    warmup_prop = 0.05
    val_bs = batch_size * 2
    
    first_epoch_eval = 1


In [ ]:
DEBUG = False
log_folder = None

In [ ]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f'Logging results to {log_folder}')
    config_df = save_config(Config, log_folder + 'config.json')
    create_logger(directory=log_folder, name="logs.txt")

meter = k_fold_det(
    Config,
    df_train,
    log_folder=log_folder
)

## Results

In [ ]:
from utils.save import load_pickle
from utils.metrics import iou_score
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
df_train = pd.read_csv(DATA_PATH + 'df_train.csv')
df_train['extended_impact'] += 1 
df_train['impact'] +=1 
folds = pd.read_csv(OUT_DIR + "folds.csv")
df_train = df_train.merge(folds, on="video")

In [ ]:
log_folder = "../logs/2020-12-15/0/"

In [ ]:
meter = load_pickle(log_folder + "meter_0.pkl")

In [ ]:
dataset = NFLDatasetDet(
    df_train.iloc[meter.val_idx],
    transforms=ToTensorV2(),
    root=IMG_PATH_F,
)

vids = list(np.unique(df_train.iloc[meter.val_idx]['video']))

In [ ]:
preds_helmet, truths_helmet, preds_impact, _ = meter.get_helmets(helmet_threshold=0.25)

In [ ]:
TOT = 0
DET = 0
PLOT = True

# for idx in np.random.choice(len(dataset), 1000):
for idx in tqdm(range(len(dataset))):
    image, boxes, label, vid, frame = dataset[idx]
    
#     if "Sideline" in vid:
#         continue
        
    boxes = boxes.int()

    boxes_pred = preds_helmet[vids.index(vid + ".mp4")]
    boxes_pred = boxes_pred[:, 1:][boxes_pred[:, 0] == frame]
    label_pred = [1] * len(boxes_pred)
    
    boxes_pred[:, 0] = boxes_pred[:, 0] * IMG_SHAPE[1] / SIZE
    boxes_pred[:, 1] = boxes_pred[:, 1] * IMG_SHAPE[0] / SIZE
    boxes_pred[:, 2] = boxes_pred[:, 2] * IMG_SHAPE[1] / SIZE
    boxes_pred[:, 3] = boxes_pred[:, 3] * IMG_SHAPE[0] / SIZE
    boxes_pred = boxes_pred.astype(int)
    
    total, detected = 0, 0
    for b in boxes[label == 2]:
        total += 1
        box = b.numpy()[np.array([1, 0, 3, 2])]
        for i in range(len(boxes_pred)):
            iou = iou_score(box, boxes_pred[i])
            if iou > 0.35:
                label_pred[i] = 2
                detected += 1
                break

    TOT += total
    DET += detected
    
    if PLOT and total != detected:
        if label.max() == 2:
            plt.figure(figsize=(15, 10))
            plt.subplot(1, 2, 1)
            plot_bboxes(image.numpy().transpose(1, 2, 0).copy(), boxes, label, transpose=True)
            plt.title('Truth')
            plt.axis(False)

            plt.subplot(1, 2, 2)
            plot_bboxes(image.numpy().transpose(1, 2, 0).copy(), boxes_pred, label_pred, transpose=False)
            plt.title('Pred')
            plt.axis(False)
            plt.show()

            print(f'Detected {detected} / {total} helmets with impacts')
    
#             break

In [ ]:
TOT, DET

### ROIs

In [ ]:
def included(box, big_box, margin=10):
    return ((box[0] - margin) > big_box[0] and 
            (box[1] - margin) > big_box[1] and 
            (box[2] + margin) < big_box[2] and 
            (box[3] + margin) < big_box[3]
           )

In [ ]:
def find_best_crop(boxes, size=512, img_shape=(720, 1280), margin=10, plot=False, image=None):
    """
    boxes : (x0, y0, x1, y1)
    """
    min_x = max(0, np.min(boxes[:, 0]) - margin)
    max_x = min(img_shape[1] - size, np.max(boxes[:, 2]) + margin)
    min_y = max(0, np.min(boxes[:, 1]) - margin)
    max_y = min(img_shape[0] - size, np.max(boxes[:, 3]) + margin)
    
    if min_x >= max_x:
        min_x = 0
        max_x = img_shape[1] - size
    if min_y >= max_y:
        min_y = 0
        max_y = img_shape[0] - size
    
    best_count = 0
    best_box = ()
    for x in range(min_x, max_x, 10):
        for y in range(min_y, max_y, 10):
            box = np.array([x, y, x + size, y + size]).astype(int)
            
            count = np.sum([included(b, box, margin=margin) for b in boxes])
            
            if count > best_count:
                best_box = box
                best_count = count
    
    if not len(best_box):
        print(min_x, max_x, min_y, max_y)
        plot_bboxes(image.numpy().transpose(1, 2, 0).copy(), list(boxes), [1] * (len(boxes)))
        plt.show()
    
    if plot and image is not None:
        plot_bboxes(image.numpy().transpose(1, 2, 0).copy(), list(boxes) + [best_box], [1] * (len(boxes) + 1))
        plt.title(best_count)
        plt.show()

    return best_box, best_count

In [ ]:
rois = []
counts = []
for idx in tqdm(range(len(dataset))):
    image, boxes, label, vid, frame = dataset[idx]

    boxes = boxes.int()

    boxes_pred = preds_helmet[vids.index(vid + ".mp4")]
    boxes_pred = boxes_pred[:, 1:][boxes_pred[:, 0] == frame]
    label_pred = [1] * len(boxes_pred)
    
    boxes_pred[:, 0] = boxes_pred[:, 0] * IMG_SHAPE[1] / SIZE
    boxes_pred[:, 1] = boxes_pred[:, 1] * IMG_SHAPE[0] / SIZE
    boxes_pred[:, 2] = boxes_pred[:, 2] * IMG_SHAPE[1] / SIZE
    boxes_pred[:, 3] = boxes_pred[:, 3] * IMG_SHAPE[0] / SIZE
    boxes_pred = boxes_pred.astype(int)
    
    roi, count = find_best_crop(boxes_pred, size=512, plot=(idx % 100) == 0, image=image)
#     roi = find_best_crop(boxes_pred, size=512, plot=idx == 289, image=image)
    rois.append(roi)
    counts.append(count)
#     break

In [ ]:
np.save(log_folder + "rois.npy", np.array(rois))
np.save(log_folder + "counts.npy", np.array(counts))

In [ ]:
sns.distplot(counts)